<a href="https://colab.research.google.com/github/crisdavid3335/analytics_with_KERAS/blob/main/series_tabulares_kaggle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

In [ ]:
data_train = np.genfromtxt('/content/drive/MyDrive/Colab Notebooks/train.csv',
                           delimiter = ',',
                           autostrip = True,
                           max_rows = 1000000,
                           dtype = np.str_)
names_full_train = np.copy(data_train[:1])
data_train = data_train[1:].astype(np.float_)

In [ ]:
data_test = np.genfromtxt('/content/drive/MyDrive/Colab Notebooks/test.csv',
                          delimiter = ',',
                          autostrip = True,
                          dtype = np.str_)
names_full_test = np.copy(data_test[:1])
data_test = data_test[1:].astype(np.float_)

In [ ]:
np.isnan(data_train).sum()

0

In [ ]:
np.isnan(data_test).sum()

0

In [ ]:
x_train = data_train
np.random.shuffle(x_train)
y_train = x_train[:,-1].astype(np.int_)

x_train = np.delete(x_train, -1, axis = 1)
x_train = np.delete(x_train, 0, axis = 1)

x_test = np.copy(data_test)
x_test = np.delete(x_test, 0, axis = 1)

In [ ]:
mean = x_train.mean(axis = 0)
x_train = x_train - mean
std = x_train.std(axis = 0)
x_train =  np.divide(x_train, std, where = std !=0)

In [ ]:
x_test = x_test - mean
x_test = np.divide(x_test,std, where = std !=0)

In [ ]:
import tensorflow.keras as keras
from keras import layers
from imblearn.under_sampling import NearMiss 

In [ ]:
np.isnan(x_test).sum()

0

In [ ]:
np.unique(y_train, return_counts = True)

(array([1, 2, 3, 4, 6, 7]),
 array([405735, 572252,   9597,     12,    624,  11779]))

In [ ]:
nm = NearMiss()
x_res, y_res = nm.fit_resample(x_train, y_train)

In [ ]:
model = keras.Sequential([
                          layers.Dense(144, activation = 'relu'),
                          layers.Dense(112, activation = 'relu'),
                          layers.Dense(80, activation = 'relu'),
                          layers.Dense(8, activation = 'softmax')
])

model.compile(optimizer = keras.optimizers.RMSprop(learning_rate = 0.0005),
              loss = 'sparse_categorical_crossentropy',
              metrics = ['accuracy'])

In [ ]:
history = model.fit(x_train, y_train, 
          batch_size = 512, 
          epochs = 30,
          callbacks = keras.callbacks.EarlyStopping(monitor = 'val_loss', 
                                                    restore_best_weights = True,                                                     
                                                    patience = 3,
                                                    verbose = 0),
          validation_split = 0.1)

Epoch 1/30
1758/1758 [==============================] - 14s 8ms/step - loss: 5.9516 - accuracy: 0.9304 - val_loss: 0.1189 - val_accuracy: 0.9520
Epoch 2/30
1758/1758 [==============================] - 13s 7ms/step - loss: 2.5045 - accuracy: 0.9533 - val_loss: 0.0980 - val_accuracy: 0.9592
Epoch 3/30
1758/1758 [==============================] - 14s 8ms/step - loss: 1.3084 - accuracy: 0.9575 - val_loss: 0.0966 - val_accuracy: 0.9593
Epoch 4/30
1758/1758 [==============================] - 13s 7ms/step - loss: 1.0280 - accuracy: 0.9598 - val_loss: 0.0950 - val_accuracy: 0.9602
Epoch 5/30
1758/1758 [==============================] - 13s 7ms/step - loss: 0.9257 - accuracy: 0.9611 - val_loss: 0.0969 - val_accuracy: 0.9587
Epoch 6/30
1758/1758 [==============================] - 13s 7ms/step - loss: 0.8823 - accuracy: 0.9618 - val_loss: 0.0832 - val_accuracy: 0.9646
Epoch 7/30
1758/1758 [==============================] - 13s 7ms/step - loss: 0.7948 - accuracy: 0.9625 - val_loss: 0.0868 - val_ac

In [ ]:
model.evaluate(x_train[:50000], y_train[:50000])

1563/1563 [==============================] - 3s 2ms/step - loss: 16.1944 - accuracy: 0.9088


[16.19440269470215, 0.9088000059127808]

In [ ]:
summit = model.predict(x_test)

In [ ]:
summit.shape

(1000000, 8)

In [ ]:
results = np.zeros((1000000,))
for i in range(0, summit.shape[0]):
  results[i] = np.argmax(summit[i])

In [ ]:
results = results.astype(np.int_)

In [ ]:
id = data_test[:, 0]

In [ ]:
import pandas as pd

submission = pd.DataFrame({"Id": id})
submission['Cover_Type'] = results
submission['Id'] = submission['Id'].astype('int32')
submission.to_csv('C:\\Users\\Christian\\Desktop\\submission.csv', index=False)